# Automatic k-points and pseudopotentials
## Run band calculations with pseudos from SSSP and k-points from SeeK-path

<div class="alert alert-box alert-info">
This example expects that you have already imported the sample data provided with the demos (to have the SSSP pseudopotential library).
</div>

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import DataFactory, CalculationFactory, Code, load_node
from aiida.orm.data.base import Str
ParameterData=DataFactory('parameter')
import os, time, numpy, pylab
import nglview
from IPython.display import Image
%matplotlib inline

#### Load the importer from the COD database, and get a silicon supercell (take the first query result)

In [ ]:
from aiida.tools.dbimporters import DbImporterFactory
CodImporter = DbImporterFactory('cod')
importer = CodImporter()

In [ ]:
all_si_results = list(importer.query(spacegroup="F d -3 m :1", formula="Si"))
cod_result = all_si_results[0] # e.g.: ID 9008565

#### Convert it to an explicit AiiDA structure, show cell and coordinates

In [ ]:
structure = cod_result.get_aiida_structure()

#### Visualize the structure

In [ ]:
view = nglview.show_ase(structure.get_ase())
view

In [ ]:
print structure._exportstring('xsf')[0]

#### Get primitive cell from SeeK-path (also standardized according to crystallographic conventions), and show some information

In [ ]:
from aiida.tools import get_kpoints_path, get_explicit_kpoints_path
seekpath_info = get_kpoints_path(structure)
primitive_structure = seekpath_info['primitive_structure']
explicit_path = get_explicit_kpoints_path(structure)

In [ ]:
parameters = seekpath_info['parameters'].get_dict()
print "Bravais lattice: {}\nSpacegroup: {}\nInput cell had a volume {}x w.r.t. the primitive cell".format(
    parameters['bravais_lattice_extended'], 
    parameters['spacegroup_international'], 
    int(parameters['volume_original_wrt_prim']))
print "Suggested path: ",
print ", ".join("{}-{}".format(p1, p2) for p1, p2 in parameters['path'])
print "K-point coordinates (scaled units):"
for label, coords in parameters['point_coords'].iteritems():
    print "{:7s} {:18.10f} {:18.10f} {:18.10f}".format(label, *coords)

#### Visualize primitive structure

In [ ]:
view = nglview.show_ase(primitive_structure.get_ase())
view

In [ ]:
Image(filename='../common/data/bz-cF2.png')

#### Get suggested cutoff from SSSP

In [ ]:
SSSP_eff_cutoffs = {l.split()[0]: int(l.split()[1]) 
                    for l in open(os.path.join(
                        os.path.dirname(__name__),os.pardir,
                        'common','data','SSSP_acc_cutoffs.txt')).readlines() 
                    if l.split()[1] != '??'}

In [ ]:
for elem in sorted(SSSP_eff_cutoffs)[:10]:
    print "{:2s}: {:3.0f} Ry".format(elem, SSSP_eff_cutoffs[elem])
print "   ..."

#### Pick the suggested cutoff for this structure, as the largest cutoff for all species in the system

In [ ]:
SSSP_cutoff = float(max(SSSP_eff_cutoffs[sym] for sym in structure.get_symbols_set()))
print "Suggested cutoff: {} Ry".format(SSSP_cutoff)

#### Pick a suitable regular mesh from a given density; here: 0.4 Angstrom<sup>-1</sup>

In [ ]:
KpointsData = DataFactory('array.kpoints')
kpts = KpointsData()
kpts.set_cell_from_structure(primitive_structure)
kpts.set_kpoints_mesh_from_density(distance=0.4, offset=[0.,0.,0.])

In [ ]:
print kpts.get_kpoints_mesh()

#### Select the code to use

In [ ]:
from notebook_helpers import get_code_pwonly_dropdown
from IPython.display import display
code_group = get_code_pwonly_dropdown()
display(code_group)

#### Generate a Quantum ESPRESSO relaxation calculation and submit it

In [ ]:
from aiida.orm import CalculationFactory, load_node
from aiida.work.run import run, submit
from aiida_quantumespresso.utils.pseudopotential import validate_and_prepare_pseudos_inputs
from aiida_quantumespresso.utils.resources import get_default_options

PwCalculation = CalculationFactory('quantumespresso.pw')

# Dictionary, where values are the code labels of each type of code required
# Here we require only PW
code_names = code_group.children[1].value
if code_names:
    code_name = code_names['pw']
    print "I will use the code '{}'".format(code_names['pw'])
else:
    print "No code found: the rest will crash. Select a code, or configure one first!"
    code_name = None
code = Code.get_from_string(code_name)
parameters = {
    'CONTROL': {
        'calculation': 'relax',
        'restart_mode': 'from_scratch',
    },
    'SYSTEM': {
        'ecutwfc': SSSP_cutoff,
        'ecutrho': 8. * SSSP_cutoff,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

inputs = {
    'code': code,
    'structure': primitive_structure,
    'kpoints': kpts,
    'parameters': ParameterData(dict=parameters),
    'pseudo': validate_and_prepare_pseudos_inputs(primitive_structure, pseudo_family=Str('SSSP')),
    '_options': get_default_options()
}

<div class="alert alert-box alert-info">
Remember at this stage to check if the daemon is started, otherwise the calculation will never run<br>
<br>
To check the daemon status, run in a terminal `verdi daemon status`<br>
To start the daemon, run in a terminal `verdi daemon start`
</div>

In [ ]:

print('Running a {} pw.x calculation... '.format('relax'))
results, pk = run(PwCalculation.process(), _return_pid=True, **inputs)
calc = load_node(pk)
print('PwCalculation<{}> terminated with state: {}'.format(calc.pk, calc.get_state()))
print('\n{link:25s} {node}'.format(link='Output link', node='Node pk and type'))
print('{s}'.format(s='-'*60))
for link, node in sorted(calc.get_outputs(also_labels=True)):
    print('{:25s} <{}> {}'.format(link, node.pk, node.__class__.__name__))

#### Perform the bands calculation 

In [ ]:
parameters = {
    'CONTROL': {
        'calculation': 'bands',
        'restart_mode': 'restart',
    },
    'SYSTEM': {
        'ecutwfc': SSSP_cutoff,
        'ecutrho': 8. * SSSP_cutoff,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

inputs = {
    'code': code,
    'structure': calc.out.output_structure,
    'parent_folder': calc.out.remote_folder,
    'kpoints': explicit_path['explicit_kpoints'],
    'parameters': ParameterData(dict=parameters),
    'pseudo': validate_and_prepare_pseudos_inputs(primitive_structure, pseudo_family=Str('SSSP')),
    '_options': get_default_options()
}

In [ ]:
print('Running a {} pw.x calculation... '.format('bands'))
results, pk = run(PwCalculation.process(), _return_pid=True, **inputs)
bandscalc = load_node(pk)
print('PwCalculation<{}> terminated with state: {}'.format(bandscalc.pk, bandscalc.get_state()))
print('\n{link:25s} {node}'.format(link='Output link', node='Node pk and type'))
print('{s}'.format(s='-'*60))
for link, node in sorted(bandscalc.get_outputs(also_labels=True)):
    print('{:25s} <{}> {}'.format(link, node.pk, node.__class__.__name__))

#### Retrieve the BandsData object and plot it with matplotlib

In [ ]:
bandsdata = bandscalc.out.output_band
bandsdata.show_mpl()